In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 46.2 MB/s eta 0:00:001m47.1 MB/s eta 0:00:01


In [3]:
import duckdb
import pandas as pd

# 1. Load Parquet from public S3 into DuckDB or pd.DataFrame
con = duckdb.connect()
con.execute("""
  SELECT trip_distance, fare_amount, passenger_count
  FROM read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet')
  LIMIT 1000
""")
df = con.fetchdf()


In [4]:
df.head()

,trip_distance,fare_amount,passenger_count
0,1.5,7.0,1.0
1,2.6,14.0,1.0
2,0.0,4.5,3.0
3,0.0,3.5,5.0
4,0.0,52.0,5.0


In [ ]:
Prepare few-shot prompt with CSV snippet

In [6]:
# 2. Prepare few-shot prompt with CSV snippet
sample_csv = df.sample(50).to_csv(index=False)
print(sample_csv)

trip_distance,fare_amount,passenger_count
16.71,47.0,6.0
4.5,20.5,2.0
0.7,5.5,1.0
1.1,7.5,1.0
1.56,8.0,1.0
1.4,10.0,1.0
2.85,15.5,1.0
0.81,5.5,1.0
2.4,10.5,1.0
0.93,6.5,1.0
0.8,5.0,1.0
1.22,9.0,2.0
0.8,5.0,4.0
1.5,6.5,1.0
1.13,5.5,2.0
1.74,10.5,5.0
1.84,12.5,2.0
2.2,10.5,1.0
1.93,10.5,5.0
0.77,6.5,1.0
0.6,5.0,1.0
4.4,18.0,1.0
2.6,12.0,2.0
1.1,6.0,3.0
16.6,48.0,4.0
2.1,10.0,2.0
0.7,7.0,1.0
2.8,15.0,2.0
3.8,19.5,1.0
1.4,5.5,1.0
0.66,5.5,1.0
1.2,7.0,1.0
1.91,10.5,2.0
1.12,6.5,1.0
0.8,5.0,3.0
4.85,15.5,1.0
9.0,37.0,1.0
4.7,15.5,2.0
3.79,14.5,2.0
0.72,7.0,1.0
0.65,4.5,1.0
1.68,7.0,1.0
5.9,24.0,2.0
1.4,7.5,1.0
0.46,4.5,1.0
1.8,7.0,1.0
3.8,14.0,2.0
0.9,5.0,1.0
5.6,18.0,2.0
0.5,3.5,1.0



In [7]:
# 2. Prepare few-shot prompt with CSV snippet
sample_csv = df.sample(50).to_csv(index=False)
prompt = f"""
You are a smart data analyst. Clean this data snippet:
- Drop rows where passenger_count is zero,
- Compute avg trip_distance and avg fare,
and output a clean summary table.

Data:
{sample_csv}
"""


In [9]:
print(prompt)


You are a smart data analyst. Clean this data snippet:
- Drop rows where passenger_count is zero,
- Compute avg trip_distance and avg fare,
and output a clean summary table.

Data:
trip_distance,fare_amount,passenger_count
1.4,7.5,2.0
0.7,4.5,1.0
8.0,27.0,1.0
3.8,16.5,2.0
3.4,12.5,1.0
1.9,9.0,3.0
3.6,15.5,3.0
1.1,6.5,1.0
1.5,8.0,1.0
1.18,6.0,1.0
5.99,18.5,4.0
3.0,12.5,1.0
0.5,4.0,2.0
0.55,6.5,1.0
2.51,10.5,1.0
0.5,4.5,2.0
3.7,13.0,1.0
0.8,6.5,1.0
0.71,6.5,2.0
0.64,5.0,1.0
1.41,6.5,1.0
4.6,22.0,1.0
0.8,4.5,1.0
3.14,14.5,1.0
0.9,6.0,1.0
0.69,5.5,3.0
1.8,8.0,1.0
0.8,5.0,3.0
1.3,9.5,2.0
1.05,5.5,1.0
0.9,6.5,2.0
0.6,5.5,3.0
1.3,19.0,1.0
1.8,11.0,3.0
0.8,9.0,4.0
8.8,27.5,1.0
1.89,9.0,2.0
0.4,3.5,3.0
3.65,15.5,3.0
1.4,7.5,3.0
4.55,17.0,1.0
3.9,13.0,1.0
2.01,10.0,2.0
0.3,4.0,2.0
4.02,15.5,1.0
0.1,2.5,2.0
4.3,19.0,2.0
9.33,28.5,1.0
3.3,18.5,1.0
1.5,7.0,1.0




In [ ]:
resp = openai.ChatCompletion.create(model="gpt-4", messages=[{"role":"user","content":prompt}])
cleaned_summary = resp.choices[0].message.content

# 3. Save summary back to S3
s3 = boto3.client('s3')
s3.put_object(Bucket='your-bucket', Key='curated/summary.txt', Body=cleaned_summary)

# 4. Question phase
qa_prompt = f"Summary:\n{cleaned_summary}\n\nQuestion: What is the average fare?"
resp2 = openai.ChatCompletion.create(model="gpt-4", messages=[{"role":"user","content":qa_prompt}])
print(resp2.choices[0].message.content)